### Estudio del rendimiento de los jugadores (análisis a partir de información extraída de los servicios en una temporada)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importams el data set
dataset = pd.read_csv('atp_matches_2023.csv')
dataset.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2023-9900,United Cup,Hard,18,A,20230102,300,126203,3.0,NaN,...,62.0,47.0,15.0,12.0,9.0,9.0,9.0,3355.0,16.0,2375.0
1,2023-9900,United Cup,Hard,18,A,20230102,299,126207,NaN,NaN,...,12.0,8.0,3.0,4.0,1.0,3.0,19.0,2000.0,23.0,1865.0
2,2023-9900,United Cup,Hard,18,A,20230102,296,126203,3.0,NaN,...,62.0,51.0,7.0,12.0,2.0,2.0,9.0,3355.0,10.0,2905.0
3,2023-9900,United Cup,Hard,18,A,20230102,295,126207,NaN,NaN,...,41.0,26.0,12.0,9.0,6.0,9.0,19.0,2000.0,245.0,220.0
4,2023-9900,United Cup,Hard,18,A,20230102,292,126774,1.0,NaN,...,58.0,48.0,18.0,16.0,1.0,2.0,4.0,5550.0,16.0,2375.0


In [3]:
jugadores = pd.concat([dataset['winner_id'], dataset['loser_id']], ignore_index=True)
codigos_jugadores = set(jugadores) # Conjunto con todos los jugadores de la temporada
players_stats = {}
for codigo in codigos_jugadores:
    iter = 0
    filas_w = dataset[(dataset['winner_id'] == codigo)]
    filas_w = filas_w.dropna(subset=['w_svpt', 'w_df', 'w_1stIn', 'w_ace', 'w_1stWon', 'w_2ndWon', 'w_bpSaved', 'w_bpFaced'])
    filas_l = dataset[(dataset['loser_id'] == codigo)]
    filas_l = filas_l.dropna(subset=['l_svpt', 'l_df', 'l_1stIn', 'l_ace', 'l_1stWon', 'l_2ndWon', 'l_bpSaved', 'l_bpFaced'])
    total = [0 for _ in range(6)]
    for result in ['w', 'l']:
        filas = filas_w if result == 'w' else filas_l
        for _, fila in filas.iterrows():
            total[0] += fila[f'{result}_1stIn'] / (fila[f'{result}_svpt'] + fila[f'{result}_df']) # Porcentaje primeros servicios
            total[1] += fila[f'{result}_ace'] / (fila[f'{result}_svpt'] + fila[f'{result}_df']) # Porcentaje de aces
            total[2] += 1 - (fila[f'{result}_df'] / (fila[f'{result}_svpt'] + fila[f'{result}_df'])) # INVERSA del porcentaje de dobles faltas
            total[3] += fila[f'{result}_1stWon'] / fila[f'{result}_1stIn'] # Porcentaje de puntos ganados en el primer sv
            total[4] += fila[f'{result}_2ndWon'] / (fila[f'{result}_svpt'] - fila[f'{result}_1stIn']) # Porcentaje de puntos ganados de en segundo sv
            if fila[f'{result}_bpFaced'] > 0:
                total[5] += fila[f'{result}_bpSaved'] / fila[f'{result}_bpFaced'] # Porcentaje de puntos de quiebre salvados
            iter += 1
    if iter > 0:
        players_stats[codigo] = [i/iter for i in total]
len(players_stats)


328

In [5]:
# Supongamos que 'columna' es la columna que quieres mirar
columna = 'winner_rank_points'

# Jugadores que no están en players_stats
jugadores_no_incluidos = codigos_jugadores - set(players_stats.keys())

# Filtrar el dataset para esos jugadores
dataset_no_incluidos = dataset[dataset['winner_id'].isin(jugadores_no_incluidos) | dataset['loser_id'].isin(jugadores_no_incluidos)]

# Mirar la columna para esos valores
dataset_no_incluidos[columna]


2857       NaN
2858    1275.0
2861     831.0
2864     820.0
2865    1835.0
         ...  
2981       NaN
2982      11.0
2983     131.0
2984      15.0
2985     131.0
Name: winner_rank_points, Length: 105, dtype: float64

In [6]:
# Supongamos que 'columna' es la columna que quieres mirar
columna = 'loser_rank_points'

# Jugadores que no están en players_stats
jugadores_no_incluidos = codigos_jugadores - set(players_stats.keys())

# Filtrar el dataset para esos jugadores
dataset_no_incluidos = dataset[dataset['winner_id'].isin(jugadores_no_incluidos) | dataset['loser_id'].isin(jugadores_no_incluidos)]

# Mirar la columna para esos valores
dataset_no_incluidos[columna]

2857    424.0
2858      4.0
2861    117.0
2864    130.0
2865     80.0
        ...  
2981      1.0
2982    190.0
2983     15.0
2984    184.0
2985     11.0
Name: loser_rank_points, Length: 105, dtype: float64